<a href="https://colab.research.google.com/github/s-sweta/Emote-Care/blob/main/new_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sairahi","key":"79735b89bc85df07bff30fffc332d575"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d msambare/fer2013

 66% 40.0M/60.3M [00:00<00:00, 58.6MB/s]
100% 60.3M/60.3M [00:00<00:00, 81.8MB/s]


In [ ]:
!kaggle datasets download -d ananthu017/emotion-detection-fer


 91% 59.0M/65.2M [00:01<00:00, 51.4MB/s]
100% 65.2M/65.2M [00:01<00:00, 55.6MB/s]


In [ ]:
!unzip emotion-detection-fer.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/sad/im37.png      
  inflating: train/sad/im370.png     
  inflating: train/sad/im3700.png    
  inflating: train/sad/im3701.png    
  inflating: train/sad/im3702.png    
  inflating: train/sad/im3703.png    
  inflating: train/sad/im3704.png    
  inflating: train/sad/im3705.png    
  inflating: train/sad/im3706.png    
  inflating: train/sad/im3707.png    
  inflating: train/sad/im3708.png    
  inflating: train/sad/im3709.png    
  inflating: train/sad/im371.png     
  inflating: train/sad/im3710.png    
  inflating: train/sad/im3711.png    
  inflating: train/sad/im3712.png    
  inflating: train/sad/im3713.png    
  inflating: train/sad/im3714.png    
  inflating: train/sad/im3715.png    
  inflating: train/sad/im3716.png    
  inflating: train/sad/im3717.png    
  inflating: train/sad/im3718.png    
  inflating: train/sad/im3719.png    
  inflating: train/sad/im372.png     
  inflating: train/sad/im3720.png    

In [ ]:
!unzip fer2013.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/sad/Training_65242339.jpg  
  inflating: train/sad/Training_65267116.jpg  
  inflating: train/sad/Training_65275626.jpg  
  inflating: train/sad/Training_6529266.jpg  
  inflating: train/sad/Training_65329617.jpg  
  inflating: train/sad/Training_65338712.jpg  
  inflating: train/sad/Training_65338797.jpg  
  inflating: train/sad/Training_65387162.jpg  
  inflating: train/sad/Training_65404494.jpg  
  inflating: train/sad/Training_65426218.jpg  
  inflating: train/sad/Training_65430136.jpg  
  inflating: train/sad/Training_65437377.jpg  
  inflating: train/sad/Training_6545735.jpg  
  inflating: train/sad/Training_65463385.jpg  
  inflating: train/sad/Training_65473985.jpg  
  inflating: train/sad/Training_65502829.jpg  
  inflating: train/sad/Training_65505359.jpg  
  inflating: train/sad/Training_65508578.jpg  
  inflating: train/sad/Training_65516023.jpg  
  inflating: train/sad/Training_65524027.jpg  
  inflating



---

# Data Augmentation

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "./train" # Directory containing the training data
test_dir = "./test"  # Directory containing the validation data

In [ ]:
import os

# Directory containing the training data
train_dir = "./train"

# Directory containing the validation data
test_dir = "./test"

def organize_data(directory):
    class_data = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            class_data[class_name] = [os.path.join(class_path, file) for file in os.listdir(class_path)]
    return class_data

# Organize training data
train_data = organize_data(train_dir)

# Organize validation data
test_data = organize_data(test_dir)

# Print the organized data
print("Training data:")
for class_name, files in train_data.items():
    print(f"{class_name}: {len(files)} files")

print("\nValidation data:")
for class_name, files in test_data.items():
    print(f"{class_name}: {len(files)} files")


Training data:
surprised: 3171 files
disgust: 436 files
fear: 4097 files
disgusted: 436 files
happy: 14430 files
neutral: 9930 files
angry: 7990 files
fearful: 4097 files
surprise: 3171 files
sad: 9660 files

Validation data:
surprised: 831 files
disgust: 111 files
fear: 1024 files
disgusted: 111 files
happy: 3548 files
neutral: 2466 files
angry: 1916 files
fearful: 1024 files
surprise: 831 files
sad: 2494 files


In [ ]:
import os

# Directory containing the training data
train_dir = "./train"

# Directory containing the validation data
test_dir = "./test"

def combine_classes(class_data):
    combined_data = {}
    for class_name, files in class_data.items():
        base_class_name = class_name.split('_')[0]  # Get the base class name
        if base_class_name not in combined_data:
            combined_data[base_class_name] = []
        combined_data[base_class_name].extend(files)
    return combined_data

def remove_classes(combined_data, classes_to_remove):
    for class_name in classes_to_remove:
        combined_data.pop(class_name, None)
    return combined_data

# Organize training data
train_data = organize_data(train_dir)

# Organize validation data
test_data = organize_data(test_dir)

# Combine classes
combined_train_data = combine_classes(train_data)
combined_test_data = combine_classes(test_data)

# Classes to remove
classes_to_remove = ['surprised', 'disgusted', 'fearful']

# Remove specified classes
combined_train_data_filtered = remove_classes(combined_train_data, classes_to_remove)
combined_test_data_filtered = remove_classes(combined_test_data, classes_to_remove)

# Print the filtered combined data
print("Filtered Combined Training data:")
for class_name, files in combined_train_data_filtered.items():
    print(f"{class_name}: {len(files)} files")

print("\nFiltered Combined Validation data:")
for class_name, files in combined_test_data_filtered.items():
    print(f"{class_name}: {len(files)} files")


Filtered Combined Training data:
disgust: 436 files
fear: 4097 files
happy: 14430 files
neutral: 9930 files
angry: 7990 files
surprise: 3171 files
sad: 9660 files

Filtered Combined Validation data:
disgust: 111 files
fear: 1024 files
happy: 3548 files
neutral: 2466 files
angry: 1916 files
surprise: 831 files
sad: 2494 files


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory containing the training data
train_dir = "./train"

# Directory containing the validation data
test_dir = "./test"

# Create a mapping of classes you want to use
class_mapping = {
    'happy': 0,
    'neutral': 1,
    'angry': 2,
    'sad': 3,
    'surprise': 4,
    'fear': 5,
    'disgust': 6,
    # Add other classes as needed
}

img_size = (48, 48)
batch_size = 64

# Use the mapping in the ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_mapping.keys(),  # Specify the classes using the keys of the mapping
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_mapping.keys(),
    shuffle=True
)


Found 49714 images belonging to 7 classes.
Found 12390 images belonging to 7 classes.




---

# Data Modelling


### CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [ ]:
#train the model
import tensorflow as tf
from tensorflow.keras import layers, models

model_cnn = models.Sequential([
    layers.Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (5, 5), padding='same'),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(512, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(512, (3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(1024),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.Dropout(0.2),

    layers.Dense(1024),
    layers.BatchNormalization(),
    layers.PReLU(alpha_initializer='zeros'),
    layers.Dropout(0.2),

    # Modify the last Dense layer to have the correct number of classes
    layers.Dense(7, activation='softmax')  # Change 10 to the actual number of classes

])

#compile the model
model_cnn.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

#callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("model_weights.h5", monitor='val_accuracy', save_weights_only=True,
                                       mode='max', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=3, verbose=1),
    tf.keras.callbacks.ModelCheckpoint("model.h5", monitor='val_loss', verbose=1, save_best_only=True)
]

In [ ]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 p_re_lu (PReLU)             (None, 48, 48, 64)        147456    
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       204928    
                                                                 
 batch_normalization_1 (Bat  (None, 24, 24, 128)       5

In [ ]:
#fit the model
history = model_cnn.fit(train_generator, epochs=25, validation_data=test_generator,
                    callbacks=callbacks)

Epoch 1/25
777/777 [==============================] - ETA: 0s - loss: 1.6129 - accuracy: 0.3833
Epoch 1: saving model to model_weights.h5

Epoch 1: val_loss improved from inf to 1.52070, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


777/777 [==============================] - 69s 75ms/step - loss: 1.6129 - accuracy: 0.3833 - val_loss: 1.5207 - val_accuracy: 0.4416 - lr: 0.0010
Epoch 2/25
777/777 [==============================] - ETA: 0s - loss: 1.2766 - accuracy: 0.5257
Epoch 2: saving model to model_weights.h5

Epoch 2: val_loss improved from 1.52070 to 1.31216, saving model to model.h5
777/777 [==============================] - 55s 71ms/step - loss: 1.2766 - accuracy: 0.5257 - val_loss: 1.3122 - val_accuracy: 0.4937 - lr: 0.0010
Epoch 3/25
777/777 [==============================] - ETA: 0s - loss: 1.1524 - accuracy: 0.5761
Epoch 3: saving model to model_weights.h5

Epoch 3: val_loss improved from 1.31216 to 1.17998, saving model to model.h5
777/777 [==============================] - 54s 70ms/step - loss: 1.1524 - accuracy: 0.5761 - val_loss: 1.1800 - val_accuracy: 0.5616 - lr: 0.0010
Epoch 4/25
777/777 [==============================] - ETA: 0s - loss: 1.0814 - accuracy: 0.6000
Epoch 4: saving model to model_wei

# Accuracy : 66.45%

In [ ]:
model_cnn.summary()